In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import  classification_report
import os

In [2]:
file = "/media/hamza/289875229874EFA4/WORK/annotation/done/txt/04_Mon-31-01-22-Fri-07-02-22/gen_ner_24k_farrukh.txt" 
df = pd.read_csv(file, header=None, delimiter = "\t")
print(df.size)
df.head()
df.to_csv('file.csv', index=False)

965


In [3]:
df = pd.read_csv('file.csv')
df.head()

,0
0,Ali Ahmed Khan
1,"AoA, guys should I upgrade to windows 11 is it..."
2,Right now it is not at the stage where you sho...
3,Muhammad Maaz Hussain i dont know why others a...
4,I upgraded to 11 the day of launch. No bugs an...


In [4]:
# mask = (df[0].str.len() > 6) & (df[0].str.len() <= 40)
# df = df.loc[mask]
# df.size

In [5]:
vect = joblib.load('data/tfidf_model.model')
# knn_model = joblib.load('data/knn_model.model')

In [6]:
def decontracted(word):
    # https://www.kaggle.com/smat26/sentiment-analysis-on-roman-urdu
    import re
    import bs4 as bs
    # import re

    word = re.sub(r'ain$', r'ein', word)
    word = re.sub(r'ai', r'ae', word)
    word = re.sub(r'ay$', r'e', word)
    word = re.sub(r'ey$', r'e', word)
    word = re.sub(r'ie$', r'y', word)
    word = re.sub(r'^es', r'is', word)
    word = re.sub(r'a+', r'a', word)
    word = re.sub(r'j+', r'j', word)
    word = re.sub(r'd+', r'd', word)
    word = re.sub(r'u', r'o', word)
    word = re.sub(r'o+', r'o', word)
    word = re.sub(r'ee+', r'i', word)
    if not re.match(r'ar', word):
        word = re.sub(r'ar', r'r', word)
        word = re.sub(r'iy+', r'i', word)
        word = re.sub(r'ih+', r'eh', word)
        word = re.sub(r's+', r's', word)
    if re.search(r'[rst]y', 'word') and word[-1] != 'y':
        word = re.sub(r'y', r'i', word)
    if re.search(r'[bcdefghijklmnopqrtuvwxyz]i', word):
        word = re.sub(r'i$', r'y', word)
    if re.search(r'[acefghijlmnoqrstuvwxyz]h', word):
        word = re.sub(r'h', '', word)
        word = re.sub(r'k', r'q', word)
        
    word = re.sub(r'\#\.', '', word)
    word = re.sub(r'\n', '', word)
    word = re.sub(r',', '', word)
    word = re.sub(r'\-', ' ', word)
    word = re.sub(r'\.', '', word)
    word = re.sub(r'\\', ' ', word)
    word = re.sub(r'\\x\.+', '', word)
    word = re.sub(r'\d', '', word)
    word = re.sub(r'^_.', '', word)
    word = re.sub(r'_', ' ', word)
    word = re.sub(r'^ ', '', word)
    word = re.sub(r' $', '', word)
    word = re.sub(r'\?', '', word)
    
    # Remove HTML tags
    word = re.sub(r"http\S+", "", word)
    word = bs.BeautifulSoup(word, 'lxml').get_text()

    # Remove numeric values including alphanueric value
    word = re.sub("\S*\d\S*", "", word).strip()

    # Remove special chars
    word = re.sub('[^A-Za-z]+', ' ', word)
    return word.lower()
    
def lemmatizing(text):
    from nltk.stem import WordNetLemmatizer
    from nltk import word_tokenize

    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    for i in range(len(tokens)):
        lemma_word = lemmatizer.lemmatize(tokens[i])
        tokens[i] = lemma_word
    return " ".join(tokens)

def text_preprocessing(df_text):
    from tqdm import tqdm
    
    preprocessed_msg = []
    # tqdm is for printing the status bar
    for sentance in tqdm(df_text.values):
        # Remove undesired chars
        sentance = decontracted(sentance)
        # sentance = lemmatizing(sentance)

        # Stop words
        ## https://deepnote.com/@umaimakhh/Sentimental-Analysis-on-Roman-Urdu-CYPLZfCxTjWGLxWzS2BaDg
        ## some words like Mein - Ma - Maa
        stopwords= ['uski', 'krna', 'kch', 'gai', 'raha', 'you', 'le', 'ki', 'krta', 'ga', 'jo', 'haan', 'de',
                    'hun', 'ho', 'meri', 'is', 'jee', 'lg', 'dain', 'hoa', 'jati', 'jana', 'rahi', 'kar', 'tou',
                    'photo', 'rha', 'ayi', 'wala', 'lye', 'kis', 'sy', 'ne', 'wohi', 'gy', 'gi', 'ny', 'hi', 'ab', 'gaya',
                    'je', 'hotay', 'jata', 'pe', 'hota', 'aye', 'me', 'tha', 'rahay', 'rhy', 'gya', 'us', 'thay', 'bhi',
                    'liye', 'sent', 'hai', 'aur', 'thi', 'jye', 'dono', 'phr', 'yehi', 'to', 'na', 'main', 'abi', 'hum',
                    'koi', 'hova', 'hain', 'ja', 'un', 'par', 'kafi', 'kr', 'hy', 'karna', 'h', 'krny', 'wo', 'kay', 'acha',
                    'mil', 'jb', 'ya', 'kya', 'ai', 'kuch', 'se', 'woh', 'thy', 'ap', 'houn', 'hay', 'sb', 'ka', 'he', 'han',
                    'chal', 'in', 'waisay', 'aese', 'hui', 'or', 'tu', 'ko', 'yeh']# , 'nai'

        
        sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)

        # Append processed sentense to list
        preprocessed_msg.append(sentance.strip())
        
    return preprocessed_msg
        
df['processed_text'] = text_preprocessing(df['0'].astype("str"))
# df.drop('0', axis=1, inplace=True)
df.head()

100%|███████████████████████████████████████| 965/965 [00:00<00:00, 1787.84it/s]


,0,processed_text
0,Ali Ahmed Khan,ali ahmed khan
1,"AoA, guys should I upgrade to windows 11 is it...",aoa goys sold i opgrade windows it wort it men...
2,Right now it is not at the stage where you sho...,rigt now it not at te stage were yo sold switc...
3,Muhammad Maaz Hussain i dont know why others a...,moammad maz hosaen i dont qnow wy oters re fac...
4,I upgraded to 11 the day of launch. No bugs an...,i opgraded te day of laonc no bogs and no lags...


In [7]:
data = vect.transform(df['processed_text'])
data = data.toarray()
print(data.shape)

(965, 2000)


In [8]:
## Testing  Boosting

# votting_model = joblib.load('data/votting_model.model')
# stacking_model = joblib.load('data/stacking_model.model')
# svm_linear_model = joblib.load("data/svm_linear.model")
# svn_nonlinear_model = joblib.load("data/svm_nonLinear.model")
# nb_model = joblib.load("data/nb.model")
lr_model = joblib.load("data/lr.model")
dt_model = joblib.load("data/dt.model")
# knn_model = joblib.load("data/knn.model")
rf_model = joblib.load("data/rf.model")

In [9]:
models = [
    # votting_model,
    #       stacking_model,
    #       svm_linear_model,
    #       svn_nonlinear_model, 
    #       nb_model, 
          lr_model, 
          dt_model,
          # knn_model, 
          rf_model]

In [10]:
predictions = []
for d in data:
    pred_list = []
    for model in models:
        for predicted in model.predict(d.reshape(1, -1)):
            pred_list.append(predicted)        
    predictions.append(max(set(pred_list), key=pred_list.count))

In [11]:
df['predictions'] = predictions

In [12]:
df.drop('processed_text', axis=1, inplace=True)
df.head()

,0,predictions
0,Ali Ahmed Khan,2
1,"AoA, guys should I upgrade to windows 11 is it...",0
2,Right now it is not at the stage where you sho...,0
3,Muhammad Maaz Hussain i dont know why others a...,0
4,I upgraded to 11 the day of launch. No bugs an...,1


In [13]:
df.to_csv('f.csv', index=False)